In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime, date

In [2]:
df = pd.read_csv('transactions_akash.csv')

In [3]:
df.columns = df.columns.str.strip()
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2022-08-19,5102600564,Quant Tax Growth Direct Plan,buy,11.771,254.8617,254.8617,2999.85
1,2022-08-19,79912985615,Mirae Asset Tax Saver Growth Direct Plan,buy,87.712,34.2010,34.2010,2999.85
2,2022-08-19,3394841/89,Tata India Tax Saving Growth Direct Plan,sell,54.654,31.8339,31.8339,1739.83
3,2022-08-19,91018778532,Axis Long Term Equity Growth Direct Plan,sell,31.521,75.7918,75.7918,2389.01
4,2022-08-19,2104123675,Invesco India Tax Growth Direct Plan,sell,18.311,90.1100,90.1100,1650.00


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
df2 = df

In [6]:
df = df[~df['Name of the Fund'].str.contains("Liquid|Money|Gilt|Cash|Bond|Debt|Short|Gold|Securities|Duration|Income")]
df = df.loc[~df['Order'].str.contains("sell")]

In [7]:
today = pd.to_datetime(date.today())

In [8]:
df = df.loc[df['Date'] > (pd.to_datetime(date.today() - timedelta(days = 1095)))]

In [9]:
mf_name = df['Name of the Fund'].unique()

In [10]:
mf_name

array(['Quant Tax Growth Direct Plan',
       'Mirae Asset Tax Saver Growth Direct Plan',
       'Bank Of India Tax Advantage Growth Direct Plan',
       'Invesco India Tax Growth Direct Plan',
       'Axis Long Term Equity Growth Direct Plan',
       'Motilal Oswal Long Term Equity Growth Direct Plan',
       'Tata India Tax Saving Growth Direct Plan',
       'PGIM India Midcap Opportunities Growth Direct Plan',
       'Quant Small Cap Growth Direct Plan',
       'DSP Healthcare Growth Direct Plan',
       'Quant Active Growth Direct Plan',
       'Nippon India Tax Saver Growth Direct Plan',
       'Franklin India Taxshield Growth Direct Plan'], dtype=object)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 635 entries, 0 to 1122
Data columns (total 8 columns):
Date                635 non-null datetime64[ns]
Folio Number        635 non-null object
Name of the Fund    635 non-null object
Order               635 non-null object
Units               635 non-null float64
NAV                 635 non-null float64
Current Nav         635 non-null float64
Amount (INR)        635 non-null float64
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 44.6+ KB


In [13]:
df['Amount (INR)'] = df['Amount (INR)']*-1

In [14]:
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2022-08-19,5102600564,Quant Tax Growth Direct Plan,buy,11.771,254.8617,254.8617,-2999.85
1,2022-08-19,79912985615,Mirae Asset Tax Saver Growth Direct Plan,buy,87.712,34.2010,34.2010,-2999.85
5,2022-08-08,5102600564,Quant Tax Growth Direct Plan,buy,37.451,248.3147,254.8617,-9299.54
6,2022-08-08,79912985615,Mirae Asset Tax Saver Growth Direct Plan,buy,281.626,33.7310,34.2010,-9499.53
23,2022-07-28,79912985615,Mirae Asset Tax Saver Growth Direct Plan,buy,214.109,32.6920,34.2010,-6999.65


In [15]:
min_rate = 0
max_rate = 0
def npv(seq_of_rates: np.array, data: pd.DataFrame) -> tuple:
    global min_rate
    global max_rate
    
    for a_rate in seq_of_rates:
        max_date = data['Date'].max()
        data['npv'] = data['Amount'] * ((1 + (a_rate / 100)) ** ((max_date - data['Date']).dt.days / 365))
        
        if data['Amount'].sum() > 0:
            if data['npv'].sum() > 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
        else:
            if data['npv'].sum() < 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
    
    return min_rate, max_rate


def xirr(data: pd.DataFrame) -> float:
    # Finding out which column contains Date and Amount
    try:
        data.iloc[:, 0].astype(float)
        col_names = ['Amount', 'Date']
    except Exception as _:
        col_names = ['Date', 'Amount']
    
    # Renaming the columns accordingly and converting to correct data types
    data.columns = col_names
    data = data.assign(Date = pd.to_datetime(arg = data['Date'], infer_datetime_format = True, dayfirst = True).dt.date,
                       Amount = data['Amount'].astype(float))
    
    if data['Amount'].sum() > 0:
        step_values = [100, 10, 1, 0.01, 0.001, 0.0001]
        stop = 10000
    else:
        step_values = [-100, -10, -1, -0.01, -0.001, -0.0001]
        stop = -10000
    
    start = 0
    for i in range(len(step_values)):
        seq_of_rates = np.arange(start = start, stop = stop, step = step_values[i])
        start, stop = npv(seq_of_rates, data)
    
    return (start + stop) / 2


In [63]:
solution = []
mf= []
for i in range(0,len(mf_name)):
    mf = df[df['Name of the Fund'] == mf_name[i]]
    mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }), 
                   ignore_index = True, sort = True)
    ready_df = mf[['Date','Amount (INR)']]
    solution.append(xirr(ready_df))

In [64]:
final_df = pd.DataFrame(list(zip(mf_name.tolist(),solution)))

In [65]:
final_df.sort_values(1, ascending = False)

,0,1
8,Quant Small Cap Growth Direct Plan,51.78235
7,PGIM India Midcap Opportunities Growth Direct ...,43.90835
10,Quant Active Growth Direct Plan,41.46145
0,Quant Tax Growth Direct Plan,32.68925
1,Mirae Asset Tax Saver Growth Direct Plan,27.05145
6,Tata India Tax Saving Growth Direct Plan,23.27885
11,Nippon India Tax Saver Growth Direct Plan,22.46055
5,Motilal Oswal Long Term Equity Growth Direct Plan,21.83305
3,Invesco India Tax Growth Direct Plan,21.82935
12,Franklin India Taxshield Growth Direct Plan,18.42105


In [66]:
final_df.sort_values(by = '0', ascending = False)

KeyError: '0'

In [ ]:
.sort_values(ascending = False)